In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
toronto = pd.read_csv('postalcodes.csv')
toronto.shape

(288, 3)

In [3]:
geodata = pd.read_csv('Geospatial_data.csv')

In [4]:
toronto = toronto[toronto.Borough != 'Not assigned']
toronto.reset_index(inplace = True, drop = True)
toronto.loc[toronto['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = toronto.loc[toronto['Neighbourhood'] == 'Not assigned', 'Borough']

In [5]:
geodata.rename(columns={'Postal Code':'Postcode'}, inplace=True)
toronto = pd.merge(toronto, geodata, on = 'Postcode', how = 'inner')
toronto.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
toronto.head(20)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353


In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
toronto = toronto.drop(columns=['Postcode'])

In [8]:
print(toronto.shape)
toronto.head()

(211, 4)


,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,Downtown Toronto,Regent Park,43.654260,-79.360636
4,North York,Lawrence Heights,43.718518,-79.464763


In [9]:
newyork = pd.read_csv('newyork.csv')

In [10]:
print(newyork.shape)
newyork.head()

(306, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
# newyork has more neighborhoods than toronto, here we randomly choose the same amount (n = 211) as toronto of neighborhoods.
newyork = newyork.sample(n = 211, random_state = 1)

In [12]:
newyork.reset_index(inplace = True, drop = True)

In [13]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.653963, -79.387207.


In [15]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [16]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WYWE4SSUQ2CZYP4ZWPGUEEBMLLRJLMV34GJSU1QWIACED2PT
CLIENT_SECRET:S2WW13KLAVIQHELPOTBUMOU1S4IVYZCCUOEDNSP0KTPOVM3Q


In [17]:
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront
Regent Park
Lawrence Heights
Lawrence Manor
Queen's Park
Islington Avenue
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design E

In [18]:
print(toronto_venues.shape)
print(toronto_venues.head())
toronto_venues.tail()

(4457, 7)
       Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0         Parkwoods              43.753259              -79.329656   
1         Parkwoods              43.753259              -79.329656   
2         Parkwoods              43.753259              -79.329656   
3  Victoria Village              43.725882              -79.315572   
4  Victoria Village              43.725882              -79.315572   

                    Venue  Venue Latitude  Venue Longitude  \
0         Brookbanks Park       43.751976       -79.332140   
1                     KFC       43.754387       -79.333021   
2           Variety Store       43.751974       -79.333114   
3  Victoria Village Arena       43.723481       -79.315635   
4             Tim Hortons       43.725517       -79.313103   

         Venue Category  
0                  Park  
1  Fast Food Restaurant  
2     Food & Drink Shop  
3          Hockey Arena  
4           Coffee Shop  


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
4452,South of Bloor,43.628841,-79.520999,Subway,43.631518,-79.518856,Sandwich Place
4453,South of Bloor,43.628841,-79.520999,Jim & Maria's No Frills,43.632004,-79.518918,Grocery Store
4454,South of Bloor,43.628841,-79.520999,Islington Florist & Nursery,43.630156,-79.518718,Flower Shop
4455,South of Bloor,43.628841,-79.520999,Kingsway Boxing Club,43.627254,-79.526684,Gym
4456,South of Bloor,43.628841,-79.520999,Burrito Boyz,43.626657,-79.526349,Burrito Place


In [19]:
newyork_venues = getNearbyVenues(names=newyork['Neighborhood'],
                                   latitudes=newyork['Latitude'],
                                   longitudes=newyork['Longitude']
                                  )

Ocean Hill
Windsor Terrace
Park Slope
Fox Hills
Lincoln Square
Prospect Heights
Hollis
Neponsit
Mill Basin
East Williamsburg
Bayswater
Cambria Heights
Hunters Point
Sandy Ground
Highland Park
Rugby
Kingsbridge Heights
Borough Park
Arlington
West Farms
Travis
Pelham Parkway
Soho
West Village
Woodhaven
Brighton Beach
Queensboro Hill
Randall Manor
Elmhurst
City Island
Greenridge
Clason Point
Arrochar
Country Club
Brookville
Central Harlem
Grant City
Starrett City
Ditmas Park
Queensbridge
Bellaire
Hillcrest
Civic Center
Prospect Lefferts Gardens
Old Town
Ozone Park
Chinatown
West Brighton
Riverdale
Hammels
Sea Gate
Oakland Gardens
Kew Gardens Hills
Williamsbridge
City Line
Fordham
Huguenot
Douglaston
Wakefield
Richmond Town
Coney Island
New Lots
Bushwick
Upper West Side
Allerton
Briarwood
Greenwich Village
Springfield Gardens
Steinway
St. George
Bay Terrace
Edgewater Park
Eltingville
Prospect Park South
Yorkville
Roxbury
East Tremont
Inwood
Flushing
Lower East Side
Pelham Gardens
Elm Park


In [20]:
print(newyork_venues.shape)
print(newyork_venues.head())
newyork_venues.tail()

(6990, 7)
  Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0   Ocean Hill              40.678403              -73.913068   
1   Ocean Hill              40.678403              -73.913068   
2   Ocean Hill              40.678403              -73.913068   
3   Ocean Hill              40.678403              -73.913068   
4   Ocean Hill              40.678403              -73.913068   

                Venue  Venue Latitude  Venue Longitude  \
0  Daily Press Coffee       40.678490       -73.910909   
1  Tower Isle Patties       40.675310       -73.913156   
2     Magic Soul Food       40.676342       -73.910610   
3      Baskin-Robbins       40.679578       -73.915990   
4         Marion Park       40.682410       -73.913144   

                    Venue Category  
0                      Coffee Shop  
1                           Bakery  
2  Southern / Soul Food Restaurant  
3                   Ice Cream Shop  
4                       Playground  


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
6985,North Riverdale,40.908543,-73.904531,C-Town Supermarkets,40.905328,-73.901056,Grocery Store
6986,North Riverdale,40.908543,-73.904531,Petite Nails,40.905375,-73.901378,Spa
6987,North Riverdale,40.908543,-73.904531,Mosholu Deli,40.905468,-73.900330,Deli / Bodega
6988,North Riverdale,40.908543,-73.904531,Hudson Parkway Pharmacy,40.905276,-73.900574,Pharmacy
6989,North Riverdale,40.908543,-73.904531,MTA MaBSTOA & Bee-Line Bus at Riverdale Ave & ...,40.912776,-73.902601,Bus Station


In [41]:
#toronto venues are saved to a csv file to avoid repeatedly calling foursquare API in the future. 
toronto_venues.to_csv('toronto_venues.csv', index = False)

In [42]:
#newyork venues are saved to a csv file to avoid repeatedly calling foursquare API in the future.
newyork_venues.to_csv('newyork_venues.csv', index = False)

In [21]:
# newyork has more venues than toronto, here we choose randomly n = 4457 venues to match the number of venues with that of toronto.
newyork_venues = newyork_venues.sample(n = 4457, random_state = 1)

In [22]:
newyork_venues.reset_index(inplace = True, drop = True)
newyork_venues.tail()

In [25]:
# one hot encoding for toronto
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
Neighborhood = toronto_onehot['Neighborhood']
toronto_onehot.drop(labels=['Neighborhood'], axis=1, inplace = True)
toronto_onehot.insert(0, 'Neighborhood', Neighborhood)

toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's 

In [ ]:
toronto_onehot.shape
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

In [28]:
toronto_grouped.shape

(205, 280)

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe for toronto
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Bar,Thai Restaurant,American Restaurant,Steakhouse,Burger Joint,Bakery,Gym,Restaurant
1,Agincourt,Chinese Restaurant,Lounge,Sandwich Place,Breakfast Spot,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,Agincourt North,Park,Playground,Asian Restaurant,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
3,Albion Gardens,Grocery Store,Liquor Store,Fried Chicken Joint,Pharmacy,Video Store,Pizza Place,Coffee Shop,Fast Food Restaurant,Beer Store,Sandwich Place
4,Alderwood,Pizza Place,Coffee Shop,Gym,Pharmacy,Skating Rink,Sandwich Place,Dance Studio,Pool,Pub,Yoga Studio


In [32]:
# one hot encoding for newyork
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newyork_onehot['Neighborhood'] = newyork_venues['Neighborhood'] 

# move neighborhood column to the first column
Neighborhood = newyork_onehot['Neighborhood']
newyork_onehot.drop(labels=['Neighborhood'], axis=1, inplace = True)
newyork_onehot.insert(0, 'Neighborhood', Neighborhood)

newyork_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Bookstore,College Gym,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Outlet Store,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Club,Roller Rink,Romanian Restaurant,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Pla

In [ ]:
print(newyork_onehot.shape)
newyork_grouped = newyork_onehot.groupby('Neighborhood').mean().reset_index()
newyork_grouped

In [34]:
# create a new dataframe for newyork
newyork_venues_sorted = pd.DataFrame(columns=columns)
newyork_venues_sorted['Neighborhood'] = newyork_grouped['Neighborhood']

for ind in np.arange(newyork_grouped.shape[0]):
    newyork_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newyork_grouped.iloc[ind, :], num_top_venues)

newyork_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pharmacy,Pizza Place,Playground,Fried Chicken Joint,Breakfast Spot,Bus Station,Spanish Restaurant,Grocery Store,Fast Food Restaurant,Chinese Restaurant
1,Arlington,Deli / Bodega,American Restaurant,Bus Stop,Grocery Store,Fruit & Vegetable Store,Food & Drink Shop,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market
2,Arrochar,Deli / Bodega,Italian Restaurant,Supermarket,Sandwich Place,Mediterranean Restaurant,Pizza Place,Liquor Store,Outdoors & Recreation,Hotel,Middle Eastern Restaurant
3,Arverne,Surf Spot,Metro Station,Board Shop,Playground,Bed & Breakfast,Thai Restaurant,Coffee Shop,Pizza Place,Sandwich Place,Beach
4,Astoria Heights,Italian Restaurant,Plaza,Music Venue,Bowling Alley,Bus Station,Shopping Mall,Bakery,Pizza Place,Hostel,Playground


In [57]:
newyork_venues_groupedwithcity = newyork_grouped
newyork_venues_groupedwithcity.insert(0, 'City', 'New York')


In [55]:
toronto_venues_groupedwithcity = toronto_grouped
toronto_venues_groupedwithcity.insert(0,'City', 'Toronto')


In [63]:
toronto_venues_groupedwithcity.shape

(205, 281)

In [64]:
newyork_venues_groupedwithcity.shape

(207, 362)

In [61]:
#Venues of two cities are combined
df_combined_venues = pd.concat([toronto_venues_groupedwithcity, newyork_venues_groupedwithcity], ignore_index=True, join='inner')

df_combined_venues.head()

,City,Neighborhood,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Toronto,Adelaide,0.0,0.0,0.04,0.0,0.01,0.01,0.0,0.020000,0.0,0.0,0.0,0.0,0.0,0.03,0.0,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.01,0.0,0.01,0.02,0.0,0.0,0.0,0.0,0.03,0.01,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.01,0.0,0.060000,0.0,0.01,0.0,0.0,0.02,0.0,0.0,0.03,0.0,0.0,0.0,0.01,0.01,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.000000,0.03,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03,0.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0,0.01,0.02,0.01,0.0,0.0,0.01,0.0,0.0,0.0,0.01,0.0,0.00

In [62]:
df_combined_venues.shape

(412, 236)

In [65]:
# set number of clusters
kclusters = 10

grouped_clustering = df_combined_venues.drop(['City','Neighborhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 7, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [ ]:
toronto_venues_sortedwithcity = toronto_venues_sorted
toronto_venues_sortedwithcity.insert(0,'City', 'Toronto')


In [76]:
newyork_venues_sortedwithcity = newyork_venues_sorted
newyork_venues_sortedwithcity.insert(0,'City', 'New York')

In [77]:
newyork_venues_sortedwithcity.head()

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New York,Allerton,Pharmacy,Pizza Place,Playground,Fried Chicken Joint,Breakfast Spot,Bus Station,Spanish Restaurant,Grocery Store,Fast Food Restaurant,Chinese Restaurant
1,New York,Arlington,Deli / Bodega,American Restaurant,Bus Stop,Grocery Store,Fruit & Vegetable Store,Food & Drink Shop,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market
2,New York,Arrochar,Deli / Bodega,Italian Restaurant,Supermarket,Sandwich Place,Mediterranean Restaurant,Pizza Place,Liquor Store,Outdoors & Recreation,Hotel,Middle Eastern Restaurant
3,New York,Arverne,Surf Spot,Metro Station,Board Shop,Playground,Bed & Breakfast,Thai Restaurant,Coffee Shop,Pizza Place,Sandwich Place,Beach
4,New York,Astoria Heights,Italian Restaurant,Plaza,Music Venue,Bowling Alley,Bus Station,Shopping Mall,Bakery,Pizza Place,Hostel,Playground


In [78]:
df_combined_venues_sorted = pd.concat([toronto_venues_sortedwithcity, newyork_venues_sortedwithcity], ignore_index=True)

# add clustering labels
df_combined_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [79]:
df_combined_venues_sorted.head(20)

,Cluster Labels,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Toronto,Adelaide,Coffee Shop,Café,Bar,Thai Restaurant,American Restaurant,Steakhouse,Burger Joint,Bakery,Gym,Restaurant
1,0,Toronto,Agincourt,Chinese Restaurant,Lounge,Sandwich Place,Breakfast Spot,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,7,Toronto,Agincourt North,Park,Playground,Asian Restaurant,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
3,0,Toronto,Albion Gardens,Grocery Store,Liquor Store,Fried Chicken Joint,Pharmacy,Video Store,Pizza Place,Coffee Shop,Fast Food Restaurant,Beer Store,Sandwich Place
4,0,Toronto,Alderwood,Pizza Place,Coffee Shop,Gym,Pharmacy,Skating Rink,Sandwich Place,Dance Studio,Pool,Pub,Yoga Studio
5,0,Toronto,Bathurst Manor,Coffee Shop,Sandwich Place,Fast Food Restaurant,Deli / Bodega,Middle Eastern Restaurant,Supermarket,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner
6,0,Toronto,Bathurst Quay,Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Coffee Shop,Plane,Bar,Boat or Ferry,Sculpture Garden,Airport Gate
7,0,Toronto,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Dim Sum Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
8,0,Toronto,Beaumond Heights,Grocery Store,Liquor Store,Fried Chicken Joint,Pharmacy,Video Store,Pizza Place,Coffee Shop,Fast Food Restaurant,Beer Store,Sandwich Place
9,0,Toronto,Bedford Park,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Indian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Restaurant,Ice Cream Shop


**Cluster 1**

In [86]:
df_combined_venues_sorted.loc[df_combined_venues_sorted['Cluster Labels'] == 0].groupby(['City']).count()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
City,,,,,,,,,,,,
New York,177,177,177,177,177,177,177,177,177,177,177,177
Toronto,162,162,162,162,162,162,162,162,162,162,162,162


**Cluster 2**

In [87]:
df_combined_venues_sorted.loc[df_combined_venues_sorted['Cluster Labels'] == 1].groupby(['City']).count()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
City,,,,,,,,,,,,
Toronto,5,5,5,5,5,5,5,5,5,5,5,5


In [97]:
df_combined_venues_sorted.loc[df_combined_venues_sorted['Cluster Labels'] == 1].head()

,Cluster Labels,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,1,Toronto,Cloverdale,Bank,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
81,1,Toronto,Islington,Bank,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
103,1,Toronto,Martin Grove,Bank,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
131,1,Toronto,Princess Gardens,Bank,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
189,1,Toronto,West Deane Park,Bank,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop


**Cluster 3**

In [88]:
df_combined_venues_sorted.loc[df_combined_venues_sorted['Cluster Labels'] == 2].groupby(['City']).count()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
City,,,,,,,,,,,,
Toronto,10,10,10,10,10,10,10,10,10,10,10,10


In [98]:
df_combined_venues_sorted.loc[df_combined_venues_sorted['Cluster Labels'] == 2]

,Cluster Labels,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,2,Toronto,Emery,Baseball Field,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Dessert Shop
72,2,Toronto,Humber Bay,Baseball Field,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Dessert Shop
76,2,Toronto,Humberlea,Baseball Field,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Dessert Shop
88,2,Toronto,King's Mill Park,Baseball Field,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Dessert Shop
90,2,Toronto,Kingsway Park South East,Baseball Field,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Dessert Shop
107,2,Toronto,Mimico NE,Baseball Field,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Dessert Shop
124,2,Toronto,Old Mill South,Baseball Field,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Dessert Shop
144,2,Toronto,Royal York South East,Baseball Field,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Dessert Shop
166,2,Toronto,Sunnylea,Baseball Field,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Dessert Shop
176,2,Toronto,The Queensway East,Baseball Field,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Dessert Shop


**Cluster 4**

In [89]:
df_combined_venues_sorted.loc[df_combined_venues_sorted['Cluster Labels'] == 3].groupby(['City']).count()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
City,,,,,,,,,,,,
New York,1,1,1,1,1,1,1,1,1,1,1,1
Toronto,3,3,3,3,3,3,3,3,3,3,3,3


In [99]:
df_combined_venues_sorted.loc[df_combined_venues_sorted['Cluster Labels'] == 3]

,Cluster Labels,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
70,3,Toronto,Highland Creek,Construction & Landscaping,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
130,3,Toronto,Port Union,Construction & Landscaping,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
143,3,Toronto,Rouge Hill,Construction & Landscaping,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
347,3,New York,Port Ivory,Bar,Yoga Studio,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food


**Cluster 5**

In [90]:
df_combined_venues_sorted.loc[df_combined_venues_sorted['Cluster Labels'] == 4].groupby(['City']).count()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
City,,,,,,,,,,,,
Toronto,2,2,2,2,2,2,2,2,2,2,2,2


In [100]:
df_combined_venues_sorted.loc[df_combined_venues_sorted['Cluster Labels'] == 4]

,Cluster Labels,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
101,4,Toronto,Malvern,Fast Food Restaurant,Print Shop,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
142,4,Toronto,Rouge,Fast Food Restaurant,Print Shop,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore


**Cluster 6**

In [91]:
df_combined_venues_sorted.loc[df_combined_venues_sorted['Cluster Labels'] == 5].groupby(['City']).count()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
City,,,,,,,,,,,,
New York,3,3,3,3,3,3,3,3,3,3,3,3
Toronto,14,14,14,14,14,14,14,14,14,14,14,14


In [101]:
df_combined_venues_sorted.loc[df_combined_venues_sorted['Cluster Labels'] == 5]

,Cluster Labels,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,5,Toronto,CFB Toronto,Park,Airport,Other Repair Shop,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
18,5,Toronto,Caledonia-Fairbanks,Park,Pharmacy,Women's Store,Fast Food Restaurant,Market,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
43,5,Toronto,Downsview East,Park,Airport,Other Repair Shop,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
49,5,Toronto,East Toronto,Park,Coffee Shop,Convenience Store,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
56,5,Toronto,Forest Hill North,Trail,Park,Sushi Restaurant,Jewelry Store,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
58,5,Toronto,Forest Hill West,Trail,Park,Sushi Restaurant,Jewelry Store,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
97,5,Toronto,Lawrence Park,Park,Bus Line,Swim School,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
110,5,Toronto,Montgomery Road,River,Park,Pool,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
123,5,Toronto,Old Mill North,River,Park,Pool,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
129,5,Toronto,Parkwoods,Fast Food Restaurant,Park,Food & Drink Shop,Ethiopian Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Department Store


**Cluster 7**

In [92]:
df_combined_venues_sorted.loc[df_combined_venues_sorted['Cluster Labels'] == 6].groupby(['City']).count()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
City,,,,,,,,,,,,
New York,20,20,20,20,20,20,20,20,20,20,20,20


In [102]:
df_combined_venues_sorted.loc[df_combined_venues_sorted['Cluster Labels'] == 6]

,Cluster Labels,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
206,6,New York,Arlington,Deli / Bodega,American Restaurant,Bus Stop,Grocery Store,Fruit & Vegetable Store,Food & Drink Shop,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market
207,6,New York,Arrochar,Deli / Bodega,Italian Restaurant,Supermarket,Sandwich Place,Mediterranean Restaurant,Pizza Place,Liquor Store,Outdoors & Recreation,Hotel,Middle Eastern Restaurant
218,6,New York,Bellaire,Deli / Bodega,Italian Restaurant,Coffee Shop,Bus Stop,Moving Target,Food,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop
225,6,New York,Briarwood,Deli / Bodega,Convenience Store,Fast Food Restaurant,Coffee Shop,Gym,Indian Restaurant,Mexican Restaurant,Food Truck,Fountain,Farmers Market
228,6,New York,Brookville,Deli / Bodega,Farm,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food
246,6,New York,Clifton,Train Station,Deli / Bodega,Bus Stop,Park,Eastern European Restaurant,Grocery Store,Fountain,Food Truck,Fruit & Vegetable Store,Filipino Restaurant
251,6,New York,Country Club,Sandwich Place,Deli / Bodega,Fried Chicken Joint,Playground,Food,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop
265,6,New York,Elm Park,Deli / Bodega,American Restaurant,Italian Restaurant,Cosmetics Shop,Ice Cream Shop,Food & Drink Shop,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop
277,6,New York,Fox Hills,Sandwich Place,Bus Stop,Yoga Studio,Food,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market
283,6,New York,Grasmere,Bus Stop,Nail Salon,Flower Shop,Bagel Shop,Bakery,Ice Cream Shop,Bank,Vegetarian / Vegan Restaurant,Deli / Bodega,Italian Restaurant


**Cluster 8**

In [93]:
df_combined_venues_sorted.loc[df_combined_venues_sorted['Cluster Labels'] == 7].groupby(['City']).count()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
City,,,,,,,,,,,,
New York,2,2,2,2,2,2,2,2,2,2,2,2
Toronto,7,7,7,7,7,7,7,7,7,7,7,7


In [103]:
df_combined_venues_sorted.loc[df_combined_venues_sorted['Cluster Labels'] == 7]

,Cluster Labels,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,7,Toronto,Agincourt North,Park,Playground,Asian Restaurant,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
92,7,Toronto,L'Amoreaux East,Park,Playground,Asian Restaurant,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
106,7,Toronto,Milliken,Park,Playground,Asian Restaurant,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
111,7,Toronto,Moore Park,Tennis Court,Playground,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
149,7,Toronto,Scarborough Village,Playground,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
160,7,Toronto,Steeles East,Park,Playground,Asian Restaurant,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
164,7,Toronto,Summerhill East,Tennis Court,Playground,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
214,7,New York,Bayswater,Tennis Court,Playground,Park,Yoga Studio,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market
294,7,New York,Holliswood,Playground,Donut Shop,Yoga Studio,Food & Drink Shop,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market


**Cluster 9**

In [94]:
df_combined_venues_sorted.loc[df_combined_venues_sorted['Cluster Labels'] == 8].groupby(['City']).count()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
City,,,,,,,,,,,,
Toronto,2,2,2,2,2,2,2,2,2,2,2,2


In [104]:
df_combined_venues_sorted.loc[df_combined_venues_sorted['Cluster Labels'] == 8]

,Cluster Labels,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
151,8,Toronto,Silver Hills,Cafeteria,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
201,8,Toronto,York Mills,Cafeteria,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


**Cluster 10**

In [95]:
df_combined_venues_sorted.loc[df_combined_venues_sorted['Cluster Labels'] == 9].groupby(['City']).count()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
City,,,,,,,,,,,,
New York,4,4,4,4,4,4,4,4,4,4,4,4


In [105]:
df_combined_venues_sorted.loc[df_combined_venues_sorted['Cluster Labels'] == 9]

,Cluster Labels,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
224,9,New York,Breezy Point,Beach,Board Shop,Monument / Landmark,Yoga Studio,Food Court,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
326,9,New York,Neponsit,Beach,Yoga Studio,Farmers Market,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food
370,9,New York,Sea Gate,Beach,Spa,Yoga Studio,Food,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market
373,9,New York,South Beach,Beach,Athletics & Sports,Yoga Studio,Food & Drink Shop,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
